# Basic Unit

离子和离子通道是神经元建模的底层模块，无论是房室的建模，还是完整神经元的建模，都离不开底层的离子与离子通道。其基本单元包括 ``Ion``、``IonChannel`` 和 ``MixIons``，从离子到通道，三者共同构建了神经元电活动的基础。

## 一、离子

### 电生理学机制
神经元的电信号源于细胞膜内外的离子浓度梯度，主要涉及：
- **钠离子（Na⁺）**：细胞外浓度高，内流引发去极化。
- **钾离子（K⁺）**：细胞内浓度高，外流导致复极化。
- **钙离子（Ca²⁺）**：既是电信号载体，也是细胞内第二信使，调控突触可塑性。

离子跨膜流动遵循Nernst方程，其驱动力由浓度差和电场共同决定：
$$
V_m = \frac{RT}{F} \ln \frac{P_{Na}[Na^+]_{out} + P_{K}[K^+]_{out}}{P_{Na}[Na^+]_{in} + P_{K}[K^+]_{in}}
$$


### 实现
通过 ``Ion`` 类定义离子的基本属性：

```python
import braincell
import brainunit as u

# 创建固定浓度的钠离子,其细胞内外浓度保持恒定
na = braincell.ion.SodiumFixed(
    size=1,
    E=50. * u.mV  # 钠反转电位，根据Nernst方程计算
)

# 设置细胞内外离子浓度
na.set_concentration(
    in_concentration=12. * u.mM,  # 细胞内浓度
    out_concentration=145. * u.mM  # 细胞外浓度
)

```

## 二、离子通道

### 电生理学机制
离子通道是镶嵌在细胞膜上的蛋白质，具有：
- **选择性通透**：特定通道只允许特定离子通过。
- **门控特性**：受电压门控、化学门控调控。
- **电导变化**：开放概率随刺激变化，决定离子电流大小：
$$
\frac{dx}{dt} = \alpha_x(V)(1-x) - \beta_x(V)x
$$


经典的Hodgkin-Huxley模型将通道动力学抽象为门控变量（如m、h、n）的变化：
$$
 \frac{dx}{dt} = \alpha_x(V)(1-x) - \beta_x(V)x
$$



### 实现

```python
import braincell
import brainunit as u

# 创建基于Hodgkin-Huxley模型的钠离子通道
ina = braincell.channel.INa_HH1952(
    size=1,  # 单神经元模拟
    g_max=120. * (u.mS / u.cm**2)  # 最大电导密度
)

# 设置通道门控参数,以下参数根据Hodgkin-Huxley原始论文
ina.set_m_params(
    alpha_A=0.1 * (u.mV**-1),  # 激活速率系数
    alpha_Vhalf=-40. * u.mV,  # 激活电压中点
    alpha_k=10. * u.mV,       # 激活电压斜率
    beta_A=4.0,               # 失活速率系数
    beta_Vhalf=-65. * u.mV,   # 失活电压中点
    beta_k=-18. * u.mV        # 失活电压斜率
)

ina.set_h_params(
    alpha_A=0.07,             # 激活速率系数
    alpha_Vhalf=-65. * u.mV,  # 激活电压中点
    alpha_k=-20. * u.mV,      # 激活电压斜率
    beta_A=1.0,               # 失活速率系数
    beta_Vhalf=-35. * u.mV,   # 失活电压中点
    beta_k=10. * u.mV         # 失活电压斜率
)

```



## 三、多离子

### 电生理学机制
真实神经元中，多种离子通道协同工作产生复杂电活动：
- **动作电位**：去极化与复极化。
- **突触整合**：兴奋性突触与抑制性突触的平衡。

混合离子系统需满足**电荷守恒**和**离子浓度平衡**，通常通过Goldman-Hodgkin-Katz方程描述：
$$
 V_m = \frac{RT}{F} \ln \frac{P_{Na}[Na^+]_{out} + P_{K}[K^+]_{out} + P_{Cl}[Cl^-]_{in}}{P_{Na}[Na^+]_{in} + P_{K}[K^+]_{in} + P_{Cl}[Cl^-]_{out}}
$$



### 实现
通过 ``MixIons`` 组合多个离子通道，构建完整模型：
```python
import braincell
import brainunit as u

# 创建钙动力学模型
ca = braincell.ion.CalciumDetailed(
    size=1,
    C_rest=5e-5 * u.mM,  # 静息钙浓度
    tau=10. * u.ms,  # 钙浓度衰减时间常数
    d=0.5 * u.um  # 钙扩散距离
)

# 创建钾离子模型
k = braincell.ion.PotassiumFixed(
    size=1,
    E=-90. * u.mV  # 钾反转电位
)

# 创建混合离子系统,包含钙离子和钾离子
kca = braincell.MixIons(k, ca)

# 添加钙激活钾通道
kca.add(
    IAHP=braincell.channel.IAHP_De1994(
        size=1,
        g_max=0.3 * (u.mS / u.cm**2)  # 最大电导密度
    )
)

```



## 四、关系

### 层级依赖关系
- ``Ion`` 是最基础单元，定义离子的物理化学属性。
- ``IonChannel`` 依赖于 ``Ion``，通过门控机制控制特定离子的流动。
- ``MixIons`` 组合多个 ``IonChannel``，整合离子电流，形成完整的膜电位动态。

### 协同工作流程
- **离子浓度梯度**:建立电化学驱动力。
- **膜电位变化**:触发离子通道门控变量改变。
- **通道电导变化**:调控离子电流大小。
- **多种离子电流叠加**:决定膜电位的最终动态。

### 生物学映射
- 单个 ``IonChannel`` 对应真实细胞膜上的离子通道蛋白。
- ``MixIons`` 对应神经元某局部膜区域的离子通道组合。
- 这种层级设计使模型可从分子水平扩展到细胞水平。
